In [9]:
# 1) Imports
import pandas as pd
from unidecode import unidecode
from pathlib import Path

In [10]:
# 2) Configuração de caminhos 

DATA_DIR = Path("data")
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Base oficial (referência)
MPM_PATH = DATA_DIR / r"C:\Users\willgnnerferreira\Documents\comparador-planilhas-serventias\Dados\Planilha-B.csv"

# Base a ser validada/atualizada
TLP_PATH = DATA_DIR / r"C:\Users\willgnnerferreira\Documents\comparador-planilhas-serventias\Dados\Planilha-A.csv"

In [11]:
# 3) Funções utilitárias (padronização + comparação)

def normalize_colname(col: str) -> str:
    """Normaliza nomes de colunas para snake_case sem acento."""
    col = unidecode(str(col)).strip().lower()
    col = col.replace("-", " ").replace("/", " ")
    col = "_".join(col.split())  # espaços -> underscore (colapsa múltiplos)
    col = col.replace("__", "_")
    return col

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [normalize_colname(c) for c in df.columns]
    # normaliza código
    if "codigo_cnj" in df.columns and "codigo_serventia" not in df.columns:
        df = df.rename(columns={"codigo_cnj": "codigo_serventia"})
    return df

def normalize_text_cols(df: pd.DataFrame, cols=("comarca", "serventia", "entrancia")) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = (
                df[c]
                .astype(str)
                .str.strip()
                .str.upper()
                .map(unidecode)
            )
            df.loc[df[c].isin(["NAN", "NONE", ""]), c] = pd.NA
    return df

def normalize_entrancia(df: pd.DataFrame) -> pd.DataFrame:
    """Tenta padronizar entrância para {INICIAL, INTERMEDIARIA, FINAL} quando fizer sentido."""
    df = df.copy()
    if "entrancia_nome" in df.columns and "entrancia" not in df.columns:
        df = df.rename(columns={"entrancia_nome": "entrancia"})

    if "entrancia" in df.columns:
        mapping = {"1": "INICIAL", "2": "INTERMEDIARIA", "3": "FINAL"}
        # se a coluna parece numérica/codificada, aplica map
        vals = df["entrancia"].dropna().astype(str).str.strip()
        if len(vals) > 0 and vals.isin(mapping.keys()).mean() > 0.8:
            df["entrancia"] = vals.map(mapping)
    return df

def read_csv_str(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(
            f"Arquivo não encontrado: {path.resolve()}\n"
            "Ajuste MPM_PATH/TLP_PATH na célula de configuração."
        )
    return pd.read_csv(path, dtype=str, encoding="utf-8", sep=",")

def compute_duplicates(df: pd.DataFrame, keys: list[str]) -> pd.DataFrame:
    if not keys:
        return df.iloc[0:0].copy()
    dup_mask = df.duplicated(subset=keys, keep=False)
    return df[dup_mask].sort_values(keys)

def build_index_dict(df: pd.DataFrame, keys: list[str]) -> dict:
    """Cria dicionário indexado pela chave (tuple)."""
    df2 = df.drop_duplicates(subset=keys).copy()
    return df2.set_index(keys).to_dict(orient="index")

def compare_reference_to_target(
    reference: pd.DataFrame,
    target: pd.DataFrame,
    keys: list[str],
    label_ref: str = "MPM",
    label_tgt: str = "TLP",
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Comparação por chave (reference -> target). Retorna (encontrados, nao_encontrados)."""
    tgt_dict = build_index_dict(target, keys)
    tem_entrancia = "entrancia" in keys

    encontrados = []
    nao_encontrados = []

    for _, linha in reference.iterrows():
        chave = tuple(linha.get(k) for k in keys)
        if chave in tgt_dict:
            encontrados.append({
                **{k: linha.get(k) for k in keys},
                "colunas_nao_correspondentes": None
            })
            continue

        # Diagnóstico simples: aponta valores ausentes no alvo
        diffs = []

        # comarca
        if "comarca" in keys and linha.get("comarca") not in set(target.get("comarca", pd.Series([], dtype=object)).dropna().unique()):
            diffs.append("comarca")

        # codigo_serventia e divergência de serventia por código
        if "codigo_serventia" in keys:
            cod = linha.get("codigo_serventia")
            if cod not in set(target.get("codigo_serventia", pd.Series([], dtype=object)).dropna().unique()):
                diffs.append("codigo_serventia")
            else:
                # código existe no alvo, mas a serventia pode divergir
                if "serventia" in keys:
                    esperadas = target[target["codigo_serventia"] == cod]["serventia"].dropna().unique()
                    if linha.get("serventia") not in set(esperadas):
                        diffs.append("serventia (diferente para mesmo codigo)")

        # entrancia
        if tem_entrancia and linha.get("entrancia") not in set(target.get("entrancia", pd.Series([], dtype=object)).dropna().unique()):
            diffs.append("entrancia")

        nao_encontrados.append({
            **{k: linha.get(k) for k in keys},
            "colunas_nao_correspondentes": ", ".join(diffs) if diffs else "CHAVE COMPLETA NAO ENCONTRADA"
        })

    return pd.DataFrame(encontrados), pd.DataFrame(nao_encontrados)

In [12]:
# 4) Leitura e normalização das bases

MPM_raw = read_csv_str(MPM_PATH)
TLP_raw = read_csv_str(TLP_PATH)

MPM = normalize_columns(MPM_raw)
TLP = normalize_columns(TLP_raw)

MPM = normalize_text_cols(MPM)
TLP = normalize_text_cols(TLP)

MPM = normalize_entrancia(MPM)
TLP = normalize_entrancia(TLP)

print("MPM colunas:", sorted(MPM.columns))
print("TLP colunas:", sorted(TLP.columns))

MPM.head()

MPM colunas: ['codigo_serventia', 'comarca', 'serventia', 'tipo_de_unidade_judiciaria']
TLP colunas: ['codigo_serventia', 'comarca', 'entrancia', 'serventia']


,comarca,serventia,codigo_serventia,tipo_de_unidade_judiciaria
0,GOIANIA,TRIBUNAL DE JUSTICA DO ESTADO DE GOIAS,16,AADJ
1,GOIANIA,1a VARA DA FAZENDA PUBLICA ESTADUAL,11381,UJ1
2,GOIANIA,2a VARA DA FAZENDA PUBLICA ESTADUAL,11382,UJ1
3,GOIANIA,1a VARA DA FAZ. PUB. MUN. E DE REG. PUB. - EXE...,11384,UJ1
4,GOIANIA,2a VARA DA FAZ. PUB. MUN. E DE REG. PUB.,11385,UJ1


In [13]:
# 5) Definição de chaves + remoção de duplicatas

keys = ["comarca", "serventia", "codigo_serventia"]
if "entrancia" in MPM.columns and "entrancia" in TLP.columns:
    keys.append("entrancia")

print("Chave usada:", keys)

# Diagnóstico de duplicatas
dup_mpm = compute_duplicates(MPM, keys)
dup_tlp = compute_duplicates(TLP, keys)

print(f"Duplicatas na MPM (pela chave): {len(dup_mpm)}")
print(f"Duplicatas na TLP (pela chave): {len(dup_tlp)}")

if len(dup_mpm) > 0:
    dup_mpm.to_csv(OUT_DIR / "duplicatas_mpm.csv", index=False, encoding="utf-8-sig")
if len(dup_tlp) > 0:
    dup_tlp.to_csv(OUT_DIR / "duplicatas_tlp.csv", index=False, encoding="utf-8-sig")

# Remover duplicatas antes da comparação
MPM_k = MPM.drop_duplicates(subset=keys).copy()
TLP_k = TLP.drop_duplicates(subset=keys).copy()

print("MPM (sem duplicatas):", MPM_k.shape)
print("TLP (sem duplicatas):", TLP_k.shape)

Chave usada: ['comarca', 'serventia', 'codigo_serventia']
Duplicatas na MPM (pela chave): 0
Duplicatas na TLP (pela chave): 0
MPM (sem duplicatas): (764, 4)
TLP (sem duplicatas): (569, 4)


In [14]:
# 6) Comparação 1: MPM → TLP (base oficial dentro da base a validar)

mpm_encontrados, mpm_nao_encontrados = compare_reference_to_target(
    reference=MPM_k,
    target=TLP_k,
    keys=keys,
    label_ref="MPM",
    label_tgt="TLP",
)

print(f"Total registros MPM: {len(MPM_k)}")
print(f"Encontrados na TLP: {len(mpm_encontrados)}")
print(f"Não encontrados na TLP: {len(mpm_nao_encontrados)}")

mpm_encontrados.head()

Total registros MPM: 764
Encontrados na TLP: 472
Não encontrados na TLP: 292


,comarca,serventia,codigo_serventia,colunas_nao_correspondentes
0,GOIANIA,1a VARA DA FAZENDA PUBLICA ESTADUAL,11381,None
1,GOIANIA,2a VARA DA FAZENDA PUBLICA ESTADUAL,11382,None
2,GOIANIA,1a VARA DA FAZ. PUB. MUN. E DE REG. PUB. - EXE...,11384,None
3,GOIANIA,2a VARA DA FAZ. PUB. MUN. E DE REG. PUB.,11385,None
4,GOIANIA,3a VARA DA FAZENDA PUBLICA ESTADUAL EXECUCAO F...,11386,None


In [15]:
# 7) Comparação 2: TLP → MPM (o que existe na TLP mas não está na base oficial)

tlp_encontrados, tlp_nao_encontrados = compare_reference_to_target(
    reference=TLP_k,
    target=MPM_k,
    keys=keys,
    label_ref="TLP",
    label_tgt="MPM",
)

print(f"Total registros TLP: {len(TLP_k)}")
print(f"Encontrados na MPM: {len(tlp_encontrados)}")
print(f"Não encontrados na MPM: {len(tlp_nao_encontrados)}")

tlp_nao_encontrados.head()

Total registros TLP: 569
Encontrados na MPM: 472
Não encontrados na MPM: 97


,comarca,serventia,codigo_serventia,colunas_nao_correspondentes
0,APARECIDA DE GOIANIA,1deg JUIZADO DE VIOLENCIA DOMESTICA E FAMILIAR...,79270,serventia (diferente para mesmo codigo)
1,TRIBUNAL DE JUSTICA,GABINETE DE JUIZ SUBSTITUTO EM 2o GRAU DR. ROG...,88722,comarca
2,TRIBUNAL DE JUSTICA,GABINETE DESA. ALICE TELES DE OLIVEIRA,87395,comarca
3,TRIBUNAL DE JUSTICA,GABINETE DESA ANA CRISTINA RIBEIRO PETERNELLA ...,84717,"comarca, serventia (diferente para mesmo codigo)"
4,TRIBUNAL DE JUSTICA,GABINETE DESA BEATRIZ FIGUEIREDO FRANCO,52639,comarca


In [16]:
# 8) Exportar resultados

mpm_encontrados.to_csv(OUT_DIR / "mpm_encontrados_na_tlp.csv", index=False, encoding="utf-8-sig")
mpm_nao_encontrados.to_csv(OUT_DIR / "mpm_nao_encontrados_na_tlp.csv", index=False, encoding="utf-8-sig")

tlp_encontrados.to_csv(OUT_DIR / "tlp_encontrados_na_mpm.csv", index=False, encoding="utf-8-sig")
tlp_nao_encontrados.to_csv(OUT_DIR / "tlp_nao_encontrados_na_mpm.csv", index=False, encoding="utf-8-sig")

print("Arquivos exportados em:", OUT_DIR.resolve())

Arquivos exportados em: C:\Users\willgnnerferreira\Documents\comparador-planilhas-serventias\outputs
